# Sweep laser pointer across the Moon from Earth

Plan: 
Use SymPy to:
1. Derive formula for distance Earth - Moon surface as a function of the sweep angle from Earth perspective
2. Derive formula for the arrival time as a function of the sweep angle under consideration of the sweep angular speed

Plot the following:
- Arrival time vs sweep angle
- Moon surface location vs sweep angle
...


In [1]:
import sympy
from sympy import *

# Constants
#x, y, s, sq = symbols('x y s sq', positive=True) #, real=True)

c = symbols('c', positive=True ) # speed of light in vacuum
R = symbols('R', positive=True ) # radius of the Moon (target sphere)
D = symbols('D', positive=True ) # Distance Earth to Moon center
alpha = symbols('alpha')         # Angle of laser pointer direction
L = symbols('L', positive=True ) # Flight path length (distance Earth - Moon surface point)
a, b = symbols('a b', positive=True ) # y and x coordinates of Moon surface point


In [20]:

geometry = solve([a*a + b*b - R*R, a - L*sin(alpha), D - b - L*cos(alpha)], L, a, b)
print( geometry[0] )
print( geometry[1] )
# select the relevant solution (must be less than D):
pathLength = geometry[0][0]
surfacePointVertical = geometry[0][1]
surfacePointHorizontal = geometry[0][2]
#print(pathLength, surfacePointVertical, surfacePointHorizontal)
pathLength

(D*cos(alpha) - sqrt(-D**2*sin(alpha)**2 + R**2), (D*cos(alpha) - sqrt(-D**2*sin(alpha)**2 + R**2))*sin(alpha), D*sin(alpha)**2 + sqrt(-D**2*sin(alpha)**2 + R**2)*cos(alpha))
(D*cos(alpha) + sqrt(-D**2*sin(alpha)**2 + R**2), (D*cos(alpha) + sqrt(-D**2*sin(alpha)**2 + R**2))*sin(alpha), D*sin(alpha)**2 - sqrt(-D**2*sin(alpha)**2 + R**2)*cos(alpha))


D*cos(alpha) - sqrt(-D**2*sin(alpha)**2 + R**2)